In [75]:
import numpy as np
import pandas as pd
from scipy.stats import gamma
from scipy.special import factorial
from scipy.special import loggamma as logg

$$
\begin{aligned}
\frac{\int f(x_{ji} \mid \phi_k)\prod_{j'i' \neq ji, z_{j'i'}=k} f(x_{j'i'} \mid \phi_k)h(\phi_k)d\phi_k}{\int \prod_{j'i' \neq ji, z_{j'i'}=k} f(x_{j'i'} \mid \phi_k)h(\phi_k)d\phi_k} &= \frac{\int\frac{\theta_k^{x_{ji}}e^{-\theta_k}}{x_{ji}!}\frac{\beta^\alpha}{\Gamma(\alpha)}\theta_k^{\alpha-1}e^{-\beta\theta_k}\prod_{j'i' \neq ji, z_{j'i'}=k} \frac{\theta_k^{x_{j'i'}}e^{-\theta_k}}{x_{j'i'}!}d\theta_k}{\int \frac{\beta^\alpha}{\Gamma(\alpha)}\theta_k^{\alpha-1}e^{-\beta\theta_k}\prod_{j'i' \neq ji, z_{j'i'}=k} \frac{\theta_k^{x_{j'i'}}e^{-\theta_k}}{x_{j'i'}!}d\theta_k} \\
&\propto \int\frac{\theta_k^{x_{ji}}e^{-\theta_k}}{x_{ji}!}\frac{\beta^\alpha}{\Gamma(\alpha)}\theta_k^{\alpha-1}e^{-\beta\theta_k}\prod_{j'i' \neq ji, z_{j'i'}=k} \frac{\theta_k^{x_{j'i'}}e^{-\theta_k}}{x_{j'i'}!}d\theta_k \\
&\propto \frac{1}{\prod_{z_{j'i'} = k} x_{j'i'}!}\frac{\beta^\alpha}{\Gamma(\alpha)}\frac{\Gamma(\alpha - 1 + \sum_{z_{j'i'} = k} x_{j'i'})}{(|V| + \beta + 1)^{\alpha - 1 + \sum_{z_{j'i'} = k} x_{j'i'}}}\int Gamma(\theta_k; \alpha - 1 + \sum_{z_{j'i'} = k} x_{j'i'}, |V| + \beta + 1)d\theta_k \\
&= \frac{1}{\prod_{z_{j'i'} = k} x_{j'i'}!}\frac{\beta^\alpha}{\Gamma(\alpha)}\frac{\Gamma(\alpha - 1 + \sum_{z_{j'i'} = k} x_{j'i'})}{(|V| + \beta + 1)^{\alpha - 1 + \sum_{z_{j'i'} = k} x_{j'i'}}}\\
\end{aligned}
$$

$$
\begin{aligned}
\frac{\int f(x_{jt} \mid \phi_k)\prod_{j't' \neq ji, z_{j't'}=k} f(x_{j't'} \mid \phi_k)h(\phi_k)d\phi_k}{\int \prod_{j't' \neq jt, z_{j't'}=k} f(x_{j't'} \mid \phi_k)h(\phi_k)d\phi_k} &= \frac{\int\prod_{j'i' \in jt}\frac{\theta_k^{x_{j'i'}}e^{-\theta_k}}{x_{j'i'}!}\frac{\beta^\alpha}{\Gamma(\alpha)}\theta_k^{\alpha-1}e^{-\beta\theta_k}\prod_{j't' \neq jt, z_{j't'}=k} \frac{\theta_k^{x_{j't'}}e^{-\theta_k}}{x_{j't'}!}d\theta_k}{\int \frac{\beta^\alpha}{\Gamma(\alpha)}\theta_k^{\alpha-1}e^{-\beta\theta_k}\prod_{j't' \neq jt, z_{j't'}=k} \frac{\theta_k^{x_{j't'}}e^{-\theta_k}}{x_{j't'}!}d\theta_k} \\
&\propto \int\prod_{j'i' \in jt}\frac{\theta_k^{x_{j'i'}}e^{-\theta_k}}{x_{j'i'}!}\frac{\beta^\alpha}{\Gamma(\alpha)}\theta_k^{\alpha-1}e^{-\beta\theta_k}\prod_{j't' \neq jt, z_{j't'}=k} \frac{\theta_k^{x_{j't'}}e^{-\theta_k}}{x_{j't'}!}d\theta_k \\
&\propto \frac{1}{\prod_{z_{j'i'} = k_{t_{jt}}} x_{j'i'}!}\frac{\beta^\alpha}{\Gamma(\alpha)}\frac{\Gamma(\alpha - 1 + \sum_{z_{j'i'} = k_{t_{jt}}} x_{j'i'})}{(|W| + \beta + |T|)^{\alpha - 1 + \sum_{z_{j'i'} = k_{t_{jt}}} x_{j'i'}}}\int Gamma(\theta_k; \alpha - 1 + \sum_{z_{j'i'} = k_{t_{jt}}} x_{j'i'}, |W| + \beta + |T|)d\theta_k \\
&= \frac{1}{\prod_{z_{j'i'} = k_{t_{jt}}} x_{j'i'}!}\frac{\beta^\alpha}{\Gamma(\alpha)}\frac{\Gamma(\alpha - 1 + \sum_{z_{j'i'} = k_{t_{jt}}} x_{j'i'})}{(|W| + \beta + |T|)^{\alpha - 1 + \sum_{z_{j'i'} = k_{t_{jt}}} x_{j'i'}}}
\end{aligned}
$$

In [68]:
DF1 = pd.DataFrame(np.c_[np.zeros(5), pd.DataFrame(np.c_[np.ones(5), np.ones(5), np.array([5,2,2,3,5])], 
                                                   columns = ["j","t","k"])], columns = ["x", "j", "t", "k"])

prob_matr = np.array([[0.1, 0.4, 0.5],[0.8, 0.1, 0.1],[0.3, 0.4, 0.3],[0.2, 0.2, 0.6],[0.3, 0.2, 0.5]])

prob_matr[DF1["k"]==2, :]

array([[0.8, 0.1, 0.1],
       [0.3, 0.4, 0.3]])

In [ ]:
def gibbs_5_1(x, j, alpha_0, gamma_0, dist_type, n_iter = 1000, burn_in=0.1):
    
    """
    Implements the Gibbs sampler from HDP paper, algorithm in 5.1
    
    parameters:
        x:         N data points (such as a word observed within a set of documents)
        j:         N labels indicating the unique collection number (such as document 7 a set of 20 documents)
        alpha_0:   prior on F
        gamma_0:   prior on H
        dist_type: distribution type
        n_iter:    number of MCMC iterations
        burn_in:   percentage of the MCMC iterations used as "burn-in" to stabilize the results
    """
    
    #initialize general objects
    N = x.shape[0]
    M = np.unique(j).shape[0]
    K = 0
    fk_ji = np.ones(N).reshape((-1,1))
    fk_jt = np.ones(M).reshape((-1,1))
    t, k = np.zeros(N), np.zeros(N)
    labels_x = pd.DataFrame(np.c_[j, t, k], columns = ["j", "t", "k"])
    
    if(dist_type == "Poisson"):
        # initialize probability model parameters
        theta_k = np.ones(K).reshape((-1,1))
        prior_matrix = np.zeros((K, 2))
        
        #gibbs sampler
        for it in range(n_iter):
            
            #update t tables
            fk_ji, theta_k, labels_x = update_fk_ji(x, fk_ji, fk_jt, theta_k, prior_matrix, labels_x, N, M, K, alpha_0, 
                                                    gamma_0, dist_type)
            M = np.unique(labels_x["t"]).shape[0]
            K = np.unique(labels_x["k"]).shape[0]
            
            #update k classes
            fk_jt, theta_k, labels_x = update_fk_jt(x, fk_ji, fk_jt, theta_k, prior_matrix, labels_x, N, M, K, alpha_0, 
                                                    gamma_0, dist_type)
            K = np.unique(labels_x["k"]).shape[0]
        
        #saving and exporting results
        mcmc_df = pd.DataFrame(np.c_[x, labels_x], columns = ["x", "j", "t", "k"])
        return(mcmc_df)
        #start_index = round(n_iter*burn_in)
        #return(mcmc_df.iloc[start_index:]) 
        

In [ ]:
#wrapper function 1 for distributions
def update_fk_ji(x, fk_ji, fk_jt, theta_k, prior_matrix, labels_x, N, M, K, alpha_0, gamma_0, dist_type):
    if(dist_type == "Poisson"):
        return(update_fk_ji_poisson(x, fk_ji, fk_jt, theta_k, prior_matrix[0], prior_matrix[1], alpha_0, gamma_0, 
                                    labels_x, N, M, K))

#wrapper function 2 for distributions
def update_fk_jt(x, fk_ji, fk_jt, theta_k, prior_matrix, labels_x, N, M, K, alpha_0, gamma_0, dist_type):
    if(dist_type == "Poisson"):
        return(update_fk_jt_poisson(x, fk_ji, fk_jt, theta_k, prior_matrix[0], prior_matrix[1], alpha_0, gamma_0, 
                                    labels_x, N, M, K))

In [ ]:
def update_fk_ji_poisson(x, fk_ji, fk_jt, theta_k, alpha, beta, alpha_0, gamma_0, labels_x, N, M, K):
    for ji in np.random.permutation(N):
        
        #set up necessary slices
        x_ji, x_fk_ji, labels_x_ji = x[ji], fk_ji[ji], labels_x.iloc[ji]
        x_other, other_fk_ji, labels_x_other = np.r_[x[:ji], x[ji+1:]], np.concatenate([fk_ji[:ji,:],fk_ji[ji+1:]],axis=0), 
                                               labels_x.iloc[pd.np.r_[:ji, ji+1:]]
        
        
        curr_prob = 0
        for k in np.unique(labels_x["k"]):
            #new table probs
            z_k = labels_x["k"]==k
            m_k = labels_x.loc[z_k].shape[0]
            x_k_other = x_other[z_k]
            log_f = -np.log(factorial(x_ji)) -np.sum(np.log(factorial(x_k_other))) + 
                    logg(x_ji + alpha + np.sum(x_k_other)) - 
                    (x_ji + alpha + np.sum(x_k_other))*np.log(1 + beta + x_k_other.shape[0]) + 
                    alpha*np.log(beta) - logg(alpha)
            
            curr_prob += m_k/(M+gamma_0)*np.exp(log_f)
            
            #existing table probs
            
        
        log_f_new = -np.log(np.math.factorial(x_ji)) + logg(x_ji + alpha) - 
                    (x_ji + alpha)*np.log(1 + beta) + alpha*np.log(beta) - logg(alpha)
        curr_prob += gamma_0/(M + gamma_0)*np.exp(log_f_new)
        t_new_prob = alpha_0*curr_prob
        
        
        
        #sampling
        
        #saving
        

In [ ]:
def update_fk_jt_poisson(x, fk_ji, fk_jt, theta_k, alpha, beta, alpha_0, gamma_0, labels_x, N, M, K):
    for jt in np.random.permutation(M):
        